In [2]:
# For Colab

# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/ICRA2024
# !pip install torchinfo

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

import sys
import math
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary

import datetime
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score

In [4]:
from ce_generator import CE5min

In [5]:
ce_train_data = np.load('./CE_dataset/ce5min_train_data.npy')
ce_train_labels = np.load('./CE_dataset/ce5min_train_labels.npy')

FileNotFoundError: [Errno 2] No such file or directory: './CE_dataset/ce5min_train_data.npy'

In [5]:

ce_test_data = np.load('./CE_dataset/ce5min_test_data.npy')
ce_test_labels = np.load('./CE_dataset/ce5min_test_labels.npy')

In [6]:
n_data = 10000
ce5 = CE5min(n_data, simple_label=False)
action_data, labels, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(labels)
print(actions)
print(action_labels)

print(len(action_data))
print(len(labels))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

ce_train_data, ce_train_labels = ce5.generate_CE_dataset()
print(ce_train_data.shape, ce_train_labels.shape)
print(ce_train_labels[np.random.randint(n_data)])

FileNotFoundError: [Errno 2] No such file or directory: './Multimodal/dataset_config.json'

In [ ]:
len(ce_train_data)

10000

In [ ]:
np.save('./CE_dataset/ce5min_train_data.npy', ce_train_data)
np.save('./CE_dataset/ce5min_train_labels.npy', ce_train_labels)
# tmp0 = ce_train_data
# tmp1 = ce_train_labels
# ce_train_data = np.load('./CE_dataset/ce5min_train_data.npy')
# ce_train_labels = np.load('./CE_dataset/ce5min_train_labels.npy')
# (ce_train_data == tmp0).all(), (ce_train_labels == tmp1).all()

(True, True)

In [ ]:
n_test_data = 1000
ce5 = CE5min(n_test_data, simple_label=False)
action_data, labels, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(labels)
print(actions)
print(action_labels)

print(len(action_data))
print(len(labels))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

ce_test_data, ce_test_labels = ce5.generate_CE_dataset()
print(ce_test_data.shape, ce_test_labels.shape)
print(ce_test_labels[np.random.randint(n_test_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'sit', 'sit', 'sit', 'sit', 'sit', 'sit', 'flush_toilet', 'wash', 'wash', 'wash', 'wash', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk']
[7, 7, 7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 4, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]
60
60
60
60
60
300
(1000, 60, 128) (1000, 60)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
np.save('./CE_dataset/ce5min_test_data.npy', ce_test_data)
np.save('./CE_dataset/ce5min_test_labels.npy', ce_test_labels)
# tmp0 = ce_train_data

In [6]:
class CEDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __getitem__(self, index):
        data = self.data[index]
        label = self.labels[index]
        return data, label

    def __len__(self):
        return len(self.data)

In [7]:
ce_train_dataset = CEDataset(ce_train_data, ce_train_labels)
ce_train_loader = DataLoader(ce_train_dataset, batch_size=256, shuffle=True, num_workers=4)

In [8]:
ce_test_dataset = CEDataset(ce_test_data, ce_test_labels)
ce_test_loader = DataLoader(ce_test_dataset, batch_size=256, shuffle=False)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Training Config

In [10]:
n_epochs = 100
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss()
# criterion = focal_loss(alpha=torch.tensor([.01, .33, .33, .33]),gamma=2, device=device)

In [11]:
def train(model, data_loader, n_epochs, lr, criterion, src_mask=None, device='cpu'):
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    model.to(device)

    summary = {'loss': [[] for _ in range(n_epochs)], 'acc': [[] for _ in range(n_epochs)]}
    for e in tqdm(range(n_epochs)):
        for i, (data, labels) in enumerate(data_loader):
            optimizer.zero_grad()
            data = data.to(device)
            labels = labels.to(device)
    #         labels = labels.squeeze(-1).to(device)

            # Run the Net
            if src_mask is not None: # For transformer
                src_mask = src_mask.to(device)
                x = model(data, src_mask)
            else:
                x = model(data)

            x = x.transpose(-1,1) # sequence cross entropy loss accepts input of dimension (N, C, L)
            print(x.shape)

            # Optimize net
            loss = criterion(x, labels)
            loss.backward()
            optimizer.step()
            summary['loss'][e].append(loss.item())

                # Calculat accuracy
            _, pred = x.data.topk(1, dim=1)
    #         print(pred.squeeze(1).shape,labels.shape)
            pred = pred.squeeze(1)
            acc = torch.sum(pred == labels)/(x.shape[0] * x.shape[-1])
            summary['acc'][e].append(acc.item())
        print(pred)
        print('Loss: {}, Accuracy: {}'.format(np.mean(summary['loss'][e]), np.mean(summary['acc'][e])))


    return summary

In [12]:
def test(model, data_loader, criterion, src_mask=None):
    model.eval()
    model.to('cpu')
    summary = {'loss': [] , 'acc': []}

    all_pred = []
    all_labels = []

    for i, (data, labels) in enumerate(tqdm(data_loader)):
    #   data = data
    #   labels = labels.to(device)
        labels = labels.squeeze(-1)

        # Run the Net
        with torch.no_grad():
            if src_mask is not None: # For transformer
                src_mask = src_mask.to('cpu')
                x = model(data, src_mask)
            else:
                x = model(data)
            x = x.transpose(-1,1)

        # Optimize net
        criterion = criterion.to('cpu')
        loss = criterion(x, labels)
        summary['loss'].append(loss.item())

            # Calculat accuracy

        _, pred = x.data.topk(1, dim=1)
    #     print(pred.shape,labels.reshape(-1).shape)
        pred = pred.squeeze(1)
    #     if max(pred[0]) > 0:

        for j in range(len(pred)):
            print("pred:",pred[j])
            print("label:", labels[j])
        acc = torch.sum(pred == labels)/(x.shape[0] * x.shape[-1])
        summary['acc'].append(acc.item())
        all_pred.append(pred.reshape(-1))
        all_labels.append(labels.reshape(-1))

    all_pred = np.concatenate(all_pred)
    all_labels = np.concatenate(all_labels)

    f1_all = f1_score(all_labels, all_pred, average='macro')
    f1_pos = f1_score(all_labels, all_pred, labels=[1,2,3], average='macro')

    precision = precision_score(all_labels, all_pred, average=None)
    recall = recall_score(all_labels, all_pred, average=None)
    precision_avg = precision_score(all_labels, all_pred, average='macro')
    recall_avg = recall_score(all_labels, all_pred, average='macro')

    print('Loss: {}, Accuracy: {}, F1_all: {}, F1_positive: {}, Precision: {}, Avg_P: {}, Recall: {}, Avg_R: {}'.format(np.mean(summary['loss']),
                                                                                                                np.mean(summary['acc']),
                                                                                                                f1_all,
                                                                                                                f1_pos,
                                                                                                                precision,
                                                                                                                precision_avg,
                                                                                                                recall,
                                                                                                                recall_avg
                                                                                                                ))

In [13]:
from typing import Optional, Sequence

import torch
from torch import Tensor
from torch import nn
from torch.nn import functional as F


class FocalLoss(nn.Module):
    """ Focal Loss, as described in https://arxiv.org/abs/1708.02002.

    It is essentially an enhancement to cross entropy loss and is
    useful for classification tasks when there is a large class imbalance.
    x is expected to contain raw, unnormalized scores for each class.
    y is expected to contain class labels.

    Shape:
        - x: (batch_size, C) or (batch_size, C, d1, d2, ..., dK), K > 0.
        - y: (batch_size,) or (batch_size, d1, d2, ..., dK), K > 0.
    """

    def __init__(self,
                 alpha: Optional[Tensor] = None,
                 gamma: float = 0.,
                 reduction: str = 'mean',
                 ignore_index: int = -100):
        """Constructor.

        Args:
            alpha (Tensor, optional): Weights for each class. Defaults to None.
            gamma (float, optional): A constant, as described in the paper.
                Defaults to 0.
            reduction (str, optional): 'mean', 'sum' or 'none'.
                Defaults to 'mean'.
            ignore_index (int, optional): class label to ignore.
                Defaults to -100.
        """
        if reduction not in ('mean', 'sum', 'none'):
            raise ValueError(
                'Reduction must be one of: "mean", "sum", "none".')

        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ignore_index = ignore_index
        self.reduction = reduction

        self.nll_loss = nn.NLLLoss(
            weight=alpha, reduction='none', ignore_index=ignore_index)

    def __repr__(self):
        arg_keys = ['alpha', 'gamma', 'ignore_index', 'reduction']
        arg_vals = [self.__dict__[k] for k in arg_keys]
        arg_strs = [f'{k}={v!r}' for k, v in zip(arg_keys, arg_vals)]
        arg_str = ', '.join(arg_strs)
        return f'{type(self).__name__}({arg_str})'

    def forward(self, x: Tensor, y: Tensor) -> Tensor:
        if x.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = x.shape[1]
            x = x.permute(0, *range(2, x.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            y = y.view(-1)

        unignored_mask = y != self.ignore_index
        y = y[unignored_mask]
        if len(y) == 0:
            return torch.tensor(0.)
        x = x[unignored_mask]

        # compute weighted cross entropy term: -alpha * log(pt)
        # (alpha is already part of self.nll_loss)
        log_p = F.log_softmax(x, dim=-1)
        ce = self.nll_loss(log_p, y)

        # get true class column from each row
        all_rows = torch.arange(len(x))
        log_pt = log_p[all_rows, y]

        # compute focal term: (1 - pt)^gamma
        pt = log_pt.exp()
        focal_term = (1 - pt)**self.gamma

        # the full loss: -alpha * ((1 - pt)^gamma) * log(pt)
        loss = focal_term * ce

        if self.reduction == 'mean':
            loss = loss.mean()
        elif self.reduction == 'sum':
            loss = loss.sum()

        return loss


def focal_loss(alpha: Optional[Sequence] = None,
               gamma: float = 0.,
               reduction: str = 'mean',
               ignore_index: int = -100,
               device='cpu',
               dtype=torch.float32) -> FocalLoss:
    """Factory function for FocalLoss.

    Args:
        alpha (Sequence, optional): Weights for each class. Will be converted
            to a Tensor if not None. Defaults to None.
        gamma (float, optional): A constant, as described in the paper.
            Defaults to 0.
        reduction (str, optional): 'mean', 'sum' or 'none'.
            Defaults to 'mean'.
        ignore_index (int, optional): class label to ignore.
            Defaults to -100.
        device (str, optional): Device to move alpha to. Defaults to 'cpu'.
        dtype (torch.dtype, optional): dtype to cast alpha to.
            Defaults to torch.float32.

    Returns:
        A FocalLoss object
    """
    if alpha is not None:
        if not isinstance(alpha, Tensor):
            alpha = torch.tensor(alpha)
        alpha = alpha.to(device=device, dtype=dtype)

    fl = FocalLoss(
        alpha=alpha,
        gamma=gamma,
        reduction=reduction,
        ignore_index=ignore_index)
    return fl

In [14]:
def Focal_loss(logits, labels, alpha=None, gamma=2):
    """Compute the focal loss between `logits` and the ground truth `labels`.
    Focal loss = -alpha_t * (1-pt)^gamma * log(pt)
    where pt is the probability of being classified to the true class.
    pt = p (if true class), otherwise pt = 1 - p. p = sigmoid(logit).
    Args:
      logits: A float tensor of size [batch, num_classes].
      labels: A float tensor of size [batch, num_classes].
      alpha: A float tensor of size [batch_size]
        specifying per-example weight for balanced cross entropy.
      gamma: A float scalar modulating loss from hard and easy examples.
    Returns:
      focal_loss: A float32 scalar representing normalized total loss.
    """
    bc_loss = F.binary_cross_entropy_with_logits(input=logits, target=labels, reduction="none")

    if gamma == 0.0:
        modulator = 1.0
    else:
        modulator = torch.exp(-gamma * labels * logits - gamma * torch.log(1 + torch.exp(-1.0 * logits)))

    loss = modulator * bc_loss

    if alpha is not None:
        weighted_loss = alpha * loss
        focal_loss = torch.sum(weighted_loss)
    else:
        focal_loss = torch.sum(loss)

    focal_loss /= torch.sum(labels)
    return focal_loss


class Loss(torch.nn.Module):
    def __init__(
        self,
        loss_type: str = "cross_entropy",
        beta: float = 0.999,
        fl_gamma=2,
        samples_per_class=None,
        class_balanced=False,
    ):
        """
        Compute the Class Balanced Loss between `logits` and the ground truth `labels`.
        Class Balanced Loss: ((1-beta)/(1-beta^n))*Loss(labels, logits)
        where Loss is one of the standard losses used for Neural Networks.

        reference: https://openaccess.thecvf.com/content_CVPR_2019/papers/Cui_Class-Balanced_Loss_Based_on_Effective_Number_of_Samples_CVPR_2019_paper.pdf

        Args:
            loss_type: string. One of "focal_loss", "cross_entropy",
                "binary_cross_entropy", "softmax_binary_cross_entropy".
            beta: float. Hyperparameter for Class balanced loss.
            fl_gamma: float. Hyperparameter for Focal loss.
            samples_per_class: A python list of size [num_classes].
                Required if class_balance is True.
            class_balanced: bool. Whether to use class balanced loss.
        Returns:
            Loss instance
        """
        super(Loss, self).__init__()

        if class_balanced is True and samples_per_class is None:
            raise ValueError("samples_per_class cannot be None when class_balanced is True")

        self.loss_type = loss_type
        self.beta = beta
        self.fl_gamma = fl_gamma
        self.samples_per_class = samples_per_class
        self.class_balanced = class_balanced

    def forward(
        self,
        logits: torch.tensor,
        labels: torch.tensor,
    ):
        """
        Compute the Class Balanced Loss between `logits` and the ground truth `labels`.
        Class Balanced Loss: ((1-beta)/(1-beta^n))*Loss(labels, logits)
        where Loss is one of the standard losses used for Neural Networks.

        Args:
            logits: A float tensor of size [batch, num_classes].
            labels: An int tensor of size [batch].
        Returns:
            cb_loss: A float tensor representing class balanced loss
        """
        if logits.ndim > 2:
            # (N, C, d1, d2, ..., dK) --> (N * d1 * ... * dK, C)
            c = logits.shape[1]
            logits = logits.permute(0, *range(2, logits.ndim), 1).reshape(-1, c)
            # (N, d1, d2, ..., dK) --> (N * d1 * ... * dK,)
            labels = labels.view(-1)
        batch_size = logits.shape[0]
        num_classes = logits.shape[1]
        labels_one_hot = F.one_hot(labels, num_classes).float()

        if self.class_balanced:
            effective_num = 1.0 - np.power(self.beta, self.samples_per_class)
            weights = (1.0 - self.beta) / np.array(effective_num)
            weights = weights / np.sum(weights) * num_classes
            weights = torch.tensor(weights, device=logits.device).float()

            if self.loss_type != "cross_entropy":
                weights = weights.unsqueeze(0)
                weights = weights.repeat(batch_size, 1) * labels_one_hot
                weights = weights.sum(1)
                weights = weights.unsqueeze(1)
                weights = weights.repeat(1, num_classes)

        else:
            weights = None

        if self.loss_type == "focal_loss":
            cb_loss = Focal_loss(logits, labels_one_hot, alpha=weights, gamma=self.fl_gamma)
        elif self.loss_type == "cross_entropy":
            cb_loss = F.cross_entropy(input=logits, target=labels_one_hot, weight=weights)
        elif self.loss_type == "binary_cross_entropy":
            cb_loss = F.binary_cross_entropy_with_logits(input=logits, target=labels_one_hot, weight=weights)
        elif self.loss_type == "softmax_binary_cross_entropy":
            pred = logits.softmax(dim=1)
            cb_loss = F.binary_cross_entropy(input=pred, target=labels_one_hot, weight=weights)
        return cb_loss

In [15]:
def stats(label, results_estimated):
    # label = np.concatenate(label, 0)
    # results_estimated = np.concatenate(results_estimated, 0)
    label_estimated = np.argmax(results_estimated, 1)
    f1 = f1_score(label, label_estimated, average='weighted')
    acc = np.sum(label == label_estimated) / label.size
    return acc, f1

# Experiment: LSTM

In [ ]:
dimension = ce_train_data.shape[-1]
print(dimension)
num_layer = 2
input_dim = dimension
hidden_dim = 128
output_dim = 4
# output_dim = 3


class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layer):
        super().__init__()
        self.lstm_net = nn.LSTM(input_size=input_dim,
                                hidden_size=hidden_dim,
                                num_layers=num_layer,
                                bidirectional=False,
                                batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x, _ = self.lstm_net(x)
#         x = x[:,-1, :] # if label is simple
        x = self.fc(x)
        logits = torch.sigmoid(x)
        return logits

128


In [ ]:
rnn_model = RNN(input_dim, hidden_dim, output_dim, num_layer)
for i, (embeds, labels) in enumerate(ce_train_loader):
    print(embeds.shape)
    out = rnn_model(embeds)
    break

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([256, 60, 128])


In [ ]:
criterion = focal_loss(alpha=torch.tensor([.005, 0.45, 0.45, 0.45]),gamma=2, device=device)
rnn_model.train()

train(model=rnn_model,
      data_loader=ce_train_loader,
      n_epochs=1000,
      lr=learning_rate,
      criterion=criterion,
      device=device
     )



  0%|          | 0/1000 [00:00<?, ?it/s]

0 he


UnboundLocalError: ignored

In [ ]:
rnn_model.eval()
rnn_model.to('cpu')
summary = {'loss': [] , 'acc': []}

all_pred = []
all_labels = []

for i, (data, labels) in enumerate(tqdm(ce_train_loader)):
#   data = data
#   labels = labels.to(device)
    labels = labels.squeeze(-1)

    # Run the Net
    with torch.no_grad():
        x = rnn_model(data)
        x = x.transpose(-1,1)

    # Optimize net
    criterion = criterion.to('cpu')
    loss = criterion(x, labels)
    summary['loss'].append(loss.item())

        # Calculat accuracy

    _, pred = x.data.topk(1, dim=1)
#     print(pred.shape,labels.reshape(-1).shape)
    pred = pred.squeeze(1)
#     if max(pred[0]) > 0:

    for j in range(len(pred)):
        print("pred:",pred[j])
        print("label:", labels[j])
    acc = torch.sum(pred == labels)/(x.shape[0] * x.shape[-1])
    summary['acc'].append(acc.item())
    all_pred.append(pred.reshape(-1))
    all_labels.append(labels.reshape(-1))

all_pred = np.concatenate(all_pred)
all_labels = np.concatenate(all_labels)

f1_weighted_all = f1_score(all_labels, all_pred, average='macro')
f1_pos_labels = f1_score(all_labels, all_pred, labels=[1,2,3], average='macro')

precision1 = precision_score(all_labels, all_pred, labels=[1,2,3], average='micro')
recall1 = recall_score(all_labels, all_pred, labels=[1,2,3], average='micro')
precision2 = precision_score(all_labels, all_pred, labels=[1,2,3], average='macro')
recall2 = recall_score(all_labels, all_pred, labels=[1,2,3], average='macro')
precision3 = precision_score(all_labels, all_pred, labels=[1,2,3], average='weighted')
recall3 = recall_score(all_labels, all_pred, labels=[1,2,3], average='weighted')
precision4 = precision_score(all_labels, all_pred, labels=[1,2,3], average=None)
recall4 = recall_score(all_labels, all_pred, labels=[1,2,3], average=None)
precision5 = precision_score(all_labels, all_pred, average=None)
recall5 = recall_score(all_labels, all_pred, average=None)

print('Loss: {}, Accuracy: {}, F1_weighted_all: {}, F1_positive_labels: {}'.format(np.mean(summary['loss']),
                                                                              np.mean(summary['acc']),
                                                                              f1_weighted_all,
                                                                              f1_pos_labels
                                                                             ))
print(precision1, recall1, precision2, recall2, precision3, recall3, precision4, recall4, precision5, recall5)

# Experiment: TCN

## Load TCN

In [18]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [19]:
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, inputs):
        """Inputs have to have dimension (N, C_in, L_in)"""
        y1 = self.tcn(inputs.transpose(-1,1))  # input should have dimension (N, C, L)
        o = self.linear(y1.transpose(-1,1)) # input to Linear layer should have dimension (N, L, C)
#         o = self.linear(y1[:, :, -1]) # for simple label
        return torch.sigmoid(o)

In [20]:
tcn_model = TCN(input_size=128, output_size=4, num_channels=[256,256,256,256,256], kernel_size=2, dropout=0.2)

print(summary(tcn_model))
print(sum(p.numel() for p in tcn_model.parameters()))

for i, (embeds, labels) in enumerate(ce_train_loader):
    out = tcn_model(embeds)
    print(out.shape)
    break

Layer (type:depth-idx)                   Param #
TCN                                      --
├─TemporalConvNet: 1-1                   --
│    └─Sequential: 2-1                   --
│    │    └─TemporalBlock: 3-1           230,656
│    │    └─TemporalBlock: 3-2           263,168
│    │    └─TemporalBlock: 3-3           263,168
│    │    └─TemporalBlock: 3-4           263,168
│    │    └─TemporalBlock: 3-5           263,168
├─Linear: 1-2                            1,028
Total params: 2,534,660
Trainable params: 2,534,660
Non-trainable params: 0
1284356
torch.Size([256, 60, 4])


In [21]:
n_epochs = 2000
criterion = focal_loss(alpha=torch.tensor([.005, 0.45, 0.45, 0.45]),gamma=2, device=device)

summary = train(model=tcn_model,
                data_loader=ce_train_loader,
                n_epochs=n_epochs,
                lr=learning_rate,
                criterion=criterion,
                device=device
                )

Output hidden; open in https://colab.research.google.com to view.

In [22]:
torch.save(tcn_model.state_dict(), 'saved_models/tcn_large.pt')

In [23]:
test(model=tcn_model,
     data_loader=ce_test_loader,
     criterion=criterion,
     src_mask=None)

  0%|          | 0/4 [00:00<?, ?it/s]

pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

 25%|██▌       | 1/4 [00:00<00:02,  1.23it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 

 50%|█████     | 2/4 [00:01<00:01,  1.40it/s]

tensor([0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

 75%|███████▌  | 3/4 [00:02<00:00,  1.52it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([2, 2, 0, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 4/4 [00:02<00:00,  1.53it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Loss: 0.0040572817088104784, Accuracy: 0.8882868587970734, F1_weighted_all: 0.43499632872116184, F1_positive_labels: 0.2662692637133059, Precision: [0.99734391 0.18540773 0.24060694 0.06558414], Recall: [0.891      0.64864865 1.         0.93113772]


In [ ]:
tcn_model.to('cpu')
summary = {'loss': [] , 'acc': []}
all_pred = []
all_labels = []

for i, (data, labels) in enumerate(tqdm(ce_train_loader)):
#   data = data
#   labels = labels.to(device)
    labels = labels.squeeze(-1)

    # Run the Net
    with torch.no_grad():
        x = tcn_model(data)
        x = x.transpose(-1,1)

    # Optimize net
    criterion = criterion.to('cpu')
    loss = criterion(x, labels)
    summary['loss'].append(loss.item())

        # Calculat accuracy

    _, pred = x.data.topk(1, dim=1)
#     print(pred.shape,labels.reshape(-1).shape)
    pred = pred.squeeze(1)
#     if max(pred[0]) > 0:

    for j in range(len(pred)):
        print("pred:",pred[j])
        print("label:", labels[j])
    acc = torch.sum(pred == labels)/(x.shape[0] * x.shape[-1])
    summary['acc'].append(acc.item())
    all_pred.append(pred.reshape(-1))
    all_labels.append(labels.reshape(-1))

all_pred = np.concatenate(all_pred)
all_labels = np.concatenate(all_labels)

f1_weighted_all = f1_score(all_labels, all_pred, average='macro')
f1_pos_labels = f1_score(all_labels, all_pred, labels=[1,2,3], average='macro')

precision1 = precision_score(all_labels, all_pred, labels=[1,2,3], average='micro')
recall1 = recall_score(all_labels, all_pred, labels=[1,2,3], average='micro')
precision2 = precision_score(all_labels, all_pred, labels=[1,2,3], average='macro')
recall2 = recall_score(all_labels, all_pred, labels=[1,2,3], average='macro')
precision3 = precision_score(all_labels, all_pred, labels=[1,2,3], average='weighted')
recall3 = recall_score(all_labels, all_pred, labels=[1,2,3], average='weighted')
precision4 = precision_score(all_labels, all_pred, labels=[1,2,3], average=None)
recall4 = recall_score(all_labels, all_pred, labels=[1,2,3], average=None)
precision5 = precision_score(all_labels, all_pred, average=None)
recall5 = recall_score(all_labels, all_pred, average=None)

print('Loss: {}, Accuracy: {}, F1_weighted_all: {}, F1_positive_labels: {}'.format(np.mean(summary['loss']),
                                                                              np.mean(summary['acc']),
                                                                              f1_weighted_all,
                                                                              f1_pos_labels
                                                                             ))
print(precision1, recall1, precision2, recall2, precision3, recall3, precision4, recall4, precision5, recall5)

# Experiment: Transformer

In [17]:
def create_src_causal_mask(sz):
    src_mask = torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1).to(torch.bool)
    return src_mask

In [18]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, output_dim, num_head=8, num_layers=6, pos_encoding=False, dropout=0.0):
        super().__init__()
        self.pos_encoding = pos_encoding
        if self.pos_encoding:
            self.pos_encoder = PositionalEncoder(d_model=input_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_head, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, src, mask=None):
        if self.pos_encoding:
            x = self.pos_encoder(src)
        x = self.transformer_encoder(src, mask=mask)  # src should have dimension (N, S, E)
        x = self.fc(x)
        logits = torch.sigmoid(x)
        return logits

In [19]:
class PositionalEncoder(torch.nn.Module):
    def __init__(self, d_model, max_seq_len=1000):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i) / d_model)))
                pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1)) / d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        with torch.no_grad():
            x = x * math.sqrt(self.d_model)
            seq_len = x.size(1)
            pe = self.pe[:, :seq_len]
            x = x + pe
            return x

In [21]:
src_causal_mask = create_src_causal_mask(ce_train_data.shape[1]) # if batch_first is True

transformer_model = TimeSeriesTransformer(input_dim=128, output_dim=4, num_head=4, num_layers=6, pos_encoding=True)

print(summary(transformer_model))
print(sum(p.numel() for p in transformer_model.parameters()))

for i, (embeds, labels) in enumerate(ce_train_loader):
    out = transformer_model(embeds, mask=src_causal_mask)
    print(embeds.shape,out.shape)
    break

Layer (type:depth-idx)                                            Param #
TimeSeriesTransformer                                             --
├─PositionalEncoder: 1-1                                          --
├─TransformerEncoderLayer: 1-2                                    --
│    └─MultiheadAttention: 2-1                                    49,536
│    │    └─NonDynamicallyQuantizableLinear: 3-1                  16,512
│    └─Linear: 2-2                                                264,192
│    └─Dropout: 2-3                                               --
│    └─Linear: 2-4                                                262,272
│    └─LayerNorm: 2-5                                             256
│    └─LayerNorm: 2-6                                             256
│    └─Dropout: 2-7                                               --
│    └─Dropout: 2-8                                               --
├─TransformerEncoder: 1-3                                         --
│    └─Mo

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


torch.Size([256, 60, 128]) torch.Size([256, 60, 4])


In [ ]:
n_epochs = 2000
learning_rate = 1e-3
# criterion = nn.CrossEntropyLoss()
criterion = focal_loss(alpha=torch.tensor([.005, 0.45, 0.45, 0.45]),gamma=2, device=device)
# criterion = focal_loss(alpha=torch.tensor([.005, .33, .33, .33]),gamma=2, device=device)
# criterion = Loss(loss_type="focal_loss",beta=0.9999,fl_gamma=2,samples_per_class=[295000,1666,1666,1666],
#         class_balanced=True)
# no_of_classes = 4
# logits = torch.rand(10,no_of_classes).float()
# labels = torch.randint(0,no_of_classes, size = (10,))
# beta = 0.9999
# gamma = 2.0
# samples_per_cls = [177,1,1,1]
# loss_type = "focal"
# cb_loss = CB_loss(labels, logits, samples_per_cls, no_of_classes,loss_type, beta, gamma)
# print(cb_loss)
# criterion = lambda x,y: CB_loss(x, y, samples_per_cls, no_of_classes,loss_type, beta, gamma).to(device)
# focal_loss(alpha=torch.tensor([.01, .33, .33, .33]),gamma=2).to(device)

summary = train(model=transformer_model,
                data_loader=ce_train_loader,
                n_epochs=n_epochs,
                lr=learning_rate,
                criterion=criterion,
                src_mask=src_causal_mask,
                device=device
               )


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
torch.save(transformer_model.state_dict(), 'saved_models/transformer.pt')

In [ ]:
test(model=transformer_model,
     data_loader=ce_test_loader,
     criterion=criterion,
     src_mask=src_causal_mask)

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:562: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._transformer_encoder_layer_fwd(


pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
        1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

 25%|██▌       | 1/4 [00:00<00:02,  1.01it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
        0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

 50%|█████     | 2/4 [00:01<00:01,  1.12it/s]

tensor([0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

 75%|███████▌  | 3/4 [00:02<00:00,  1.18it/s]

tensor([0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 4/4 [00:03<00:00,  1.17it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 3, 3, 0, 3, 3, 3, 3, 3, 3,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
transformer_model.eval()
transformer_model.to('cpu')
summary = {'loss': [] , 'acc': []}

all_pred = []
all_labels = []

for i, (data, labels) in enumerate(tqdm(ce_train_loader)):
#   data = data
#   labels = labels.to(device)
    labels = labels.squeeze(-1)

    # Run the Net
    with torch.no_grad():
        x = transformer_model(data)
        x = x.transpose(-1,1)

    # Optimize net
    criterion = criterion.to('cpu')
    loss = criterion(x, labels)
    summary['loss'].append(loss.item())

        # Calculat accuracy
    _, pred = x.data.topk(1, dim=1)
#         print(pred.squeeze(1).shape,labels.shape)
    pred = pred.squeeze(1)
#     if max(pred[0]) > 0:
    for j in range(len(pred)):
        print("pred:",pred[j])
        print("label:", labels[j])
    acc = torch.sum(pred == labels)/(x.shape[0] * x.shape[-1])
    summary['acc'].append(acc.item())
    all_pred.append(pred.reshape(-1))
    all_labels.append(labels.reshape(-1))

all_pred = np.concatenate(all_pred)
all_labels = np.concatenate(all_labels)

f1_weighted_all = f1_score(all_labels, all_pred, average='macro')
f1_pos_labels = f1_score(all_labels, all_pred, labels=[1,2,3], average='macro')

precision1 = precision_score(all_labels, all_pred, labels=[1,2,3], average='micro')
recall1 = recall_score(all_labels, all_pred, labels=[1,2,3], average='micro')
precision2 = precision_score(all_labels, all_pred, labels=[1,2,3], average='macro')
recall2 = recall_score(all_labels, all_pred, labels=[1,2,3], average='macro')
precision3 = precision_score(all_labels, all_pred, labels=[1,2,3], average='weighted')
recall3 = recall_score(all_labels, all_pred, labels=[1,2,3], average='weighted')
precision4 = precision_score(all_labels, all_pred, labels=[1,2,3], average=None)
recall4 = recall_score(all_labels, all_pred, labels=[1,2,3], average=None)
precision5 = precision_score(all_labels, all_pred, average=None)
recall5 = recall_score(all_labels, all_pred, average=None)

print('Loss: {}, Accuracy: {}, F1_weighted_all: {}, F1_positive_labels: {}'.format(np.mean(summary['loss']),
                                                                              np.mean(summary['acc']),
                                                                              f1_weighted_all,
                                                                              f1_pos_labels
                                                                             ))
print(precision1, recall1, precision2, recall2, precision3, recall3, precision4, recall4, precision5, recall5)

In [ ]:
ce_train_dataset.labels[-111]

# Neural + FSM

In [ ]:
import json
!pip install wavio
from multimodal import *


# get Multimodal Fusion Embedding Dataset for training and testing
config_file = './Multimodal/dataset_config.json'
with open(config_file, 'r') as f:
    dataset_config = json.load(f)

fusion_embed_train_dataset = np.load('./Multimodal/fusion_5_audio1234_imu1234_embeddings.npz', allow_pickle=True)
fusion_embed_train_set = FusionEmbed(fusion_embed_train_dataset, dataset_config)

fusion_embed_test_dataset = np.load('./Multimodal/fusion_5_audio5_imu5_embeddings.npz', allow_pickle=True)
fusion_embed_test_set = FusionEmbed(fusion_embed_test_dataset, dataset_config)

In [ ]:
fusion_embed_train_loader = DataLoader(fusion_embed_train_set, batch_size=128,
                            shuffle=True, num_workers=4)
fusion_embed_test_loader = DataLoader(fusion_embed_test_set, batch_size=128,
                            shuffle=False, num_workers=4)

In [ ]:
class FusionClassifier(nn.Module):
    def __init__(
        self,
        fusion_dim,
        output_dim,
        dropout_p=0.0,
        ):
        super().__init__()
        self.fc = nn.Linear(
            in_features=fusion_dim,
            out_features=output_dim
            )
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, fusion_embeddings):
        logits = self.fc(fusion_embeddings)
        logits = torch.sigmoid(logits)
        return logits


In [ ]:
fusion_dim = 128
n_class = dataset_config['nClasses']

fusion_classifier = FusionClassifier(fusion_dim,
                                    n_class
                                   )

print(summary(fusion_classifier))

Layer (type:depth-idx)                   Param #
FusionClassifier                         --
├─Linear: 1-1                            1,161
├─Dropout: 1-2                           --
Total params: 1,161
Trainable params: 1,161
Non-trainable params: 0


In [ ]:
fusion_classifier.load_state_dict(torch.load('saved_models/AE_classifier.pt'))

<All keys matched successfully>

## Train the AE classifier

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(fusion_classifier.parameters(), lr=0.001, momentum=0.9)

# Place on GPU
fusion_classifier.train()
fusion_classifier.to(device)

# Training loop
n_epochs = 50
summary = {'loss': [[] for _ in range(n_epochs)], 'acc': [[] for _ in range(n_epochs)]}
for e in range(n_epochs):
    for i, (fusion_embeds, labels) in enumerate(tqdm(fusion_embed_train_loader)):
        # Zero the grads
        optimizer.zero_grad()
        fusion_embeds = fusion_embeds.to(device)
        labels = labels.to(device)

        # Run the Net
        x = fusion_classifier(fusion_embeds)
#         print(x.shape)
        # print(labels.shape)
        # x = x.view(x.size()[:-1])

        # Optimize net
        loss = criterion(x, labels.long())
        loss.backward()
        optimizer.step()
        summary['loss'][e].append(loss.item())

            # Calculate accuracy
        _, pred = x.data.topk(1, dim=1)
        pred = pred.view(pred.shape[:-1])
        acc = torch.sum(pred == labels)/x.shape[0]
        summary['acc'][e].append(acc.item())

    print('Epoch: {}, Loss: {}, Accuracy: {}'.format(e, np.mean(summary['loss'][e]), np.mean(summary['acc'][e])))

100%|██████████| 29/29 [00:00<00:00, 56.07it/s]


Epoch: 0, Loss: 2.1812212878260118, Accuracy: 0.11099137931034483


100%|██████████| 29/29 [00:00<00:00, 53.14it/s]


Epoch: 1, Loss: 2.13382914970661, Accuracy: 0.18211206896551724


100%|██████████| 29/29 [00:00<00:00, 55.74it/s]


...

Epoch: 47, Loss: 1.5487492865529553, Accuracy: 0.9380387931034483


100%|██████████| 29/29 [00:00<00:00, 52.84it/s]


Epoch: 48, Loss: 1.5465358947885448, Accuracy: 0.9369612068965517


100%|██████████| 29/29 [00:00<00:00, 60.41it/s]

Epoch: 49, Loss: 1.5461720557048404, Accuracy: 0.9299568965517241


In [ ]:
torch.save(fusion_classifier.state_dict(), 'saved_models/AE_classifier.pt')

In [ ]:
fusion_classifier = FusionClassifier(fusion_dim,
                                    n_class
                                   )
fusion_classifier.load_state_dict(torch.load('saved_models/AE_classifier.pt'))

FusionClassifier(
  (fc): Linear(in_features=128, out_features=9, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)

## Test the AE classifier

In [ ]:
criterion = nn.CrossEntropyLoss()
fusion_classifier.eval()
fusion_classifier.to('cpu')

test_loss = []
test_acc = []

for i, (embeds, labels) in enumerate(fusion_embed_test_loader):

    # Run the Net
    x = fusion_classifier(embeds)
    # Optimize net
    loss = criterion(x, labels.long())
    test_loss.append(loss.item())

        # Calculat accuracy
    _, pred = x.data.topk(1, dim=1)
    pred = pred.view(pred.shape[:-1])
    acc = torch.sum(pred == labels)/x.shape[0]
    test_acc.append(acc.item())

print('Loss: {}, Accuracy: {}'.format(np.mean(test_loss), np.mean(test_acc)))

Loss: 1.5932004153728485, Accuracy: 0.9140625


In [ ]:
ce_train_data.shape, ce_train_labels.shape

((10000, 60, 128), (10000, 60))

In [ ]:
print(dataset_config['classes'][0])

brush_teeth


In [ ]:
from fsm import *

In [ ]:
windows = 60
all_preds = []
all_labels = []

class_list = dataset_config['classes']

for i, (ce_embeds, ce_labels) in enumerate(ce_train_loader):
    preds = []
    for t in range(windows):
        embeds = ce_embeds[:,t,:]
        # Run the Net
        x = fusion_classifier(embeds)
        # Optimize net

            # Calculat accuracy
        _, pred = x.data.topk(1, dim=1)
        preds.append(pred)

    preds = np.concatenate(preds, axis=1)

    all_preds.append(preds)
    all_labels.append(ce_labels)


all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

len(all_preds), len(all_labels), all_preds[0].shape, all_labels[0].shape


(10000, 10000, (60,), (60,))

In [ ]:
all_ce_labels = []

for i in range(len(all_preds)):
    preds = all_preds[i]
    fsm_list = [Event1FSM(), Event2FSM(), Event3FSM()]
    ce_labels = []

    for j in range(len(preds)):
        action_id = preds[j]
        action = class_list[action_id]


        ce_label = 0
        for fsm in fsm_list:
            l = fsm.update_state(input=action)
            if l > 0: ce_label = l
        ce_labels.append(ce_label)

    ce_labels = np.array(ce_labels)
    all_ce_labels.append(ce_labels)

all_ce_labels = np.concatenate(all_ce_labels)
all_true_labels = all_labels.reshape(-1)
print(all_true_labels.shape)


(600000,)


In [ ]:
f1_weighted_all = f1_score(all_true_labels, all_ce_labels, average='macro')
f1_pos_labels = f1_score(all_true_labels, all_ce_labels, labels=[1,2,3], average='macro')

precision1 = precision_score(all_true_labels, all_ce_labels, labels=[1,2,3], average='micro')
recall1 = recall_score(all_true_labels, all_ce_labels, labels=[1,2,3], average='micro')
precision2 = precision_score(all_true_labels, all_ce_labels, labels=[1,2,3], average='macro')
recall2 = recall_score(all_true_labels, all_ce_labels, labels=[1,2,3], average='macro')
precision3 = precision_score(all_true_labels, all_ce_labels, labels=[1,2,3], average='weighted')
recall3 = recall_score(all_true_labels, all_ce_labels, labels=[1,2,3], average='weighted')
precision4 = precision_score(all_true_labels, all_ce_labels, labels=[1,2,3], average=None)
recall4 = recall_score(all_true_labels, all_ce_labels, labels=[1,2,3], average=None)
precision5 = precision_score(all_true_labels, all_ce_labels, average=None)
recall5 = recall_score(all_true_labels, all_ce_labels, average=None)

print('Accuracy: {}, F1_weighted_all: {}, F1_positive_labels: {}'.format(np.mean(summary['acc']),
                                                                         f1_weighted_all,
                                                                         f1_pos_labels
                                                                        ))
print(precision1, recall1, precision2, recall2, precision3, recall3, precision4, recall4, precision5, recall5)

Accuracy: 0.9041864224137931, F1_weighted_all: 0.8744834072612377, F1_positive_labels: 0.8333333333333334
0.959844244341689 0.7888 0.9290780141843972 0.7887788778877888 0.9290851063829787 0.7888 [0.78723404 1.         1.        ] [0.36633663 1.         1.        ] [0.99643112 0.78723404 1.         1.        ] [0.99944068 0.36633663 1.         1.        ]


# Generate CE data with one-hot AE
The AE labels are not ground-truth labels, instead they are the predicted AEs by the AE classifier.

In [ ]:
fusion_classifier = FusionClassifier(fusion_dim,
                                    n_class
                                   )
fusion_classifier.load_state_dict(torch.load('saved_models/AE_classifier.pt'))

<All keys matched successfully>

In [ ]:
windows = 60
ae2ce_train_data = []
ae2ce_train_labels = []

class_list = dataset_config['classes']


for i, (ce_embeds, ce_labels) in enumerate(ce_train_loader):
    preds_one_hot = []
    for t in range(windows):
        embeds = ce_embeds[:,t,:]
        # Run the Net
        x = fusion_classifier(embeds)
        # Optimize net

            # Calculat accuracy
        _, pred = x.data.topk(1, dim=1)

        pred_one_hot = torch.nn.functional.one_hot(torch.squeeze(pred, -1), num_classes=9)
        pred_one_hot = torch.unsqueeze(pred_one_hot, 1)

        preds_one_hot.append(pred_one_hot)

    preds_one_hot = np.concatenate(preds_one_hot, axis=1)
    ae2ce_train_data.append(preds_one_hot)
    ae2ce_train_labels.append(ce_labels)


ae2ce_train_data = np.concatenate(ae2ce_train_data).astype('float32')
ae2ce_train_labels = np.concatenate(ae2ce_train_labels)

ae2ce_train_data.shape, ae2ce_train_labels.shape, ae2ce_train_data[0], ae2ce_train_labels[0]


((10000, 60, 9),
 (10000, 60),
 array([[0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
   

In [ ]:
windows = 60
ae2ce_test_data = []
ae2ce_test_labels = []

class_list = dataset_config['classes']


for i, (ce_embeds, ce_labels) in enumerate(ce_test_loader):
    preds_one_hot = []
    for t in range(windows):
        embeds = ce_embeds[:,t,:]
        # Run the Net
        x = fusion_classifier(embeds)
        # Optimize net

            # Calculat accuracy
        _, pred = x.data.topk(1, dim=1)

        pred_one_hot = torch.nn.functional.one_hot(torch.squeeze(pred, -1), num_classes=9)
        pred_one_hot = torch.unsqueeze(pred_one_hot, 1)

        preds_one_hot.append(pred_one_hot)

    preds_one_hot = np.concatenate(preds_one_hot, axis=1)
    ae2ce_test_data.append(preds_one_hot)
    ae2ce_test_labels.append(ce_labels)


ae2ce_test_data = np.concatenate(ae2ce_test_data).astype('float32')
ae2ce_test_labels = np.concatenate(ae2ce_test_labels)

ae2ce_test_data.shape, ae2ce_test_labels.shape, ae2ce_test_data[0], ae2ce_test_labels[0]


((1000, 60, 9),
 (1000, 60),
 array([[0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
     

In [ ]:
np.save('./CE_dataset/ae2ce5min_train_data.npy', ae2ce_train_data)
np.save('./CE_dataset/ae2ce5min_train_labels.npy', ae2ce_train_labels)

In [ ]:
np.save('./CE_dataset/ae2ce5min_test_data.npy', ae2ce_test_data)
np.save('./CE_dataset/ae2ce5min_test_labels.npy', ae2ce_test_labels)

In [26]:
ae2ce_train_data = np.load('./CE_dataset/ae2ce5min_train_data.npy')
ae2ce_train_labels = np.load('./CE_dataset/ae2ce5min_train_labels.npy')
ae2ce_test_data = np.load('./CE_dataset/ae2ce5min_test_data.npy')
ae2ce_test_labels = np.load('./CE_dataset/ae2ce5min_test_labels.npy')

In [27]:
ae2ce_train_dataset = CEDataset(ae2ce_train_data, ae2ce_train_labels)
ae2ce_train_loader = DataLoader(ae2ce_train_dataset, batch_size=256, shuffle=True, num_workers=4)
ae2ce_test_dataset = CEDataset(ae2ce_test_data, ae2ce_test_labels)
ae2ce_test_loader = DataLoader(ae2ce_test_dataset, batch_size=256, shuffle=False)

# Experiment: Neural + LSTM

In [ ]:
dimension = ae2ce_train_data.shape[-1]
print(dimension)
num_layer = 2
input_dim = dimension
hidden_dim = 128
output_dim = 4
# output_dim = 3


class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layer):
        super().__init__()
        self.lstm_net = nn.LSTM(input_size=input_dim,
                                hidden_size=hidden_dim,
                                num_layers=num_layer,
                                bidirectional=False,
                                batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x, _ = self.lstm_net(x)
#         x = x[:,-1, :] # if label is simple
        x = self.fc(x)
        logits = torch.sigmoid(x)
        return logits

9


In [ ]:
ae_rnn_model = RNN(input_dim, hidden_dim, output_dim, num_layer)
for i, (embeds, labels) in enumerate(ae2ce_train_loader):
    print(embeds.shape)
    out = ae_rnn_model(embeds)
    break

torch.Size([256, 60, 9])


In [ ]:
criterion = focal_loss(alpha=torch.tensor([.005, 0.45, 0.45, 0.45]),gamma=2, device=device)
ae_rnn_model.train()

summary = train(model=ae_rnn_model,
                data_loader=ae2ce_train_loader,
                n_epochs=2000,
                lr=learning_rate,
                criterion=criterion,
                device=device
                )


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
test(model=ae_rnn_model,
     data_loader=ae2ce_test_loader,
     criterion=criterion,
     src_mask=None)

  0%|          | 0/4 [00:00<?, ?it/s]

pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

 25%|██▌       | 1/4 [00:00<00:01,  2.21it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

 50%|█████     | 2/4 [00:01<00:01,  1.84it/s]

...

 75%|███████▌  | 3/4 [00:01<00:00,  1.94it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 4/4 [00:02<00:00,  1.98it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Experiment: Neural + TCN

In [28]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [29]:
class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)

    def forward(self, inputs):
        """Inputs have to have dimension (N, C_in, L_in)"""
        y1 = self.tcn(inputs.transpose(-1,1))  # input should have dimension (N, C, L)
        o = self.linear(y1.transpose(-1,1)) # input to Linear layer should have dimension (N, L, C)
#         o = self.linear(y1[:, :, -1]) # for simple label
        return torch.sigmoid(o)

In [30]:
input_dim = ae2ce_train_data.shape[-1]
ae_tcn_model = TCN(input_size=input_dim, output_size=4, num_channels=[256,256,256,256,256], kernel_size=2, dropout=0.2)

print(summary(ae_tcn_model))
print(sum(p.numel() for p in ae_tcn_model.parameters()))
for i, (embeds, labels) in enumerate(ae2ce_train_loader):
    out = ae_tcn_model(embeds)
    print(out.shape)
    break

TypeError: ignored

In [33]:
ae_tcn_model = TCN(input_size=input_dim, output_size=4, num_channels=[256,256,256,256,256], kernel_size=2, dropout=0.2)

ae_tcn_model.load_state_dict(torch.load('saved_models/ae_tcn.pt'))

<All keys matched successfully>

In [33]:
n_epochs = 2000
criterion = focal_loss(alpha=torch.tensor([.005, 0.45, 0.45, 0.45]),gamma=2, device=device)

summary = train(model=ae_tcn_model,
                data_loader=ae2ce_train_loader,
                n_epochs=n_epochs,
                lr=learning_rate,
                criterion=criterion,
                device=device
                )


Output hidden; open in https://colab.research.google.com to view.

In [34]:
torch.save(ae_tcn_model.state_dict(), 'saved_models/ae_tcn.pt')

In [34]:
test(model=ae_tcn_model,
     data_loader=ae2ce_test_loader,
     criterion=criterion,
     src_mask=None)

  0%|          | 0/4 [00:00<?, ?it/s]

pred: tensor([0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

 25%|██▌       | 1/4 [00:00<00:02,  1.44it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 

 50%|█████     | 2/4 [00:01<00:01,  1.40it/s]

...

 75%|███████▌  | 3/4 [00:02<00:00,  1.30it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 4/4 [00:02<00:00,  1.39it/s]

tensor([0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Experiment: Neural + Transformer

In [22]:
def create_src_causal_mask(sz):
    src_mask = torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1).to(torch.bool)
    return src_mask

In [23]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, output_dim, num_head=8, num_layers=6, pos_encoding=False, dropout=0.0):
        super().__init__()
        self.pos_encoding = pos_encoding
        if self.pos_encoding:
            self.pos_encoder = PositionalEncoder(d_model=input_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_head, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, src, mask=None):
        if self.pos_encoding:
            x = self.pos_encoder(src)
        x = self.transformer_encoder(src, mask=mask)  # src should have dimension (N, S, E)
        x = self.fc(x)
        logits = torch.sigmoid(x)
        return logits

In [24]:
class PositionalEncoder(torch.nn.Module):
    def __init__(self, d_model, max_seq_len=1000):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i) / d_model)))
                if i < d_model - 1:
                    pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1)) / d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        with torch.no_grad():
            x = x * math.sqrt(self.d_model)
            seq_len = x.size(1)
            pe = self.pe[:, :seq_len]
            x = x + pe
            return x

In [28]:
input_dim = ae2ce_train_data.shape[-1]

src_causal_mask = create_src_causal_mask(ae2ce_train_data.shape[1]) # if batch_first is True

ae_transformer_model = TimeSeriesTransformer(input_dim=input_dim, output_dim=4, num_head=1, num_layers=6, pos_encoding=True)

print(summary(ae_transformer_model))
print(sum(p.numel() for p in ae_transformer_model.parameters()))

for i, (embeds, labels) in enumerate(ae2ce_train_loader):
    out = ae_transformer_model(embeds, mask=src_causal_mask)
    print(embeds.shape,out.shape)
    break

Layer (type:depth-idx)                                            Param #
TimeSeriesTransformer                                             --
├─PositionalEncoder: 1-1                                          --
├─TransformerEncoderLayer: 1-2                                    --
│    └─MultiheadAttention: 2-1                                    270
│    │    └─NonDynamicallyQuantizableLinear: 3-1                  90
│    └─Linear: 2-2                                                20,480
│    └─Dropout: 2-3                                               --
│    └─Linear: 2-4                                                18,441
│    └─LayerNorm: 2-5                                             18
│    └─LayerNorm: 2-6                                             18
│    └─Dropout: 2-7                                               --
│    └─Dropout: 2-8                                               --
├─TransformerEncoder: 1-3                                         --
│    └─ModuleList: 2

In [ ]:
n_epochs = 2000
learning_rate = 1e-3
# criterion = nn.CrossEntropyLoss()
criterion = focal_loss(alpha=torch.tensor([.005, 0.45, 0.45, 0.45]),gamma=2, device=device)
summary = train(model=ae_transformer_model,
                data_loader=ae2ce_train_loader,
                n_epochs=n_epochs,
                lr=learning_rate,
                criterion=criterion,
                src_mask=src_causal_mask,
                device=device
               )


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
torch.save(ae_transformer_model.state_dict(), 'saved_models/ae_transformer.pt')

In [ ]:
test(model=ae_transformer_model,
     data_loader=ae2ce_test_loader,
     criterion=criterion,
     src_mask=src_causal_mask)

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(


pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

 25%|██▌       | 1/4 [00:00<00:02,  1.31it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

 50%|█████     | 2/4 [00:01<00:01,  1.28it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

 75%|███████▌  | 3/4 [00:02<00:00,  1.16it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

100%|██████████| 4/4 [00:03<00:00,  1.17it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
pred: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 